In [5]:
df.to_parquet('my_file.parquet', engine='pyarrow')

In [1]:
from prefect import flow, task 
from prefect_gcp.cloud_storage import GcsBucket
from prefect_gcp import GcpCredentials
from prefect_gcp.bigquery import GcpCredentials, BigQueryWarehouse
import os
from urllib.parse import urlencode
from bs4 import BeautifulSoup as bs
import requests
from datetime import datetime, timedelta
import re
import pandas as pd
import numpy as np
import random
import time
from yaml import safe_load
import json 
import argparse


In [2]:
def extract_rightmove_url(
    locationIdentifier="REGION^87490", 
    index=0,
    propertyTypes="bungalow,detached,flat,park-home,semi-detached,terraced", 
    maxDaysSinceAdded=1, 
    keywords="",
) -> str:
    """Generates a rightmove query URL from optional parameters. The default is set to:
    
        locationIdentifier is set to London
        index is set to 0, which represents the first page of results
        propertyTypes set to view bungalows, detached, flats, park-homes, semi-detached and terraced homes
        maxDaysSinceAdded set to 1, which is properties added in the last day
        keywords defaults to empty"""

    results_per_page=48
    
    maxDaysSinceAdded_valid = {'',1,3,7,14}
    if maxDaysSinceAdded not in maxDaysSinceAdded_valid:
        raise ValueError("results: maxDaysSinceAdded must be one of %r." % maxDaysSinceAdded_valid)

    params = {
        "locationIdentifier": locationIdentifier,
        "index": index,
        "propertyTypes": propertyTypes,
        "maxDaysSinceAdded": maxDaysSinceAdded,
        "numberOfPropertiesPerPage": results_per_page,
        "keywords": keywords,
    }

    # Generate the URL from the parameters given
    url = "https://www.rightmove.co.uk/property-for-sale/find.html?" + urlencode(params)
    return url

url = extract_rightmove_url()
url

'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E87490&index=0&propertyTypes=bungalow%2Cdetached%2Cflat%2Cpark-home%2Csemi-detached%2Cterraced&maxDaysSinceAdded=1&numberOfPropertiesPerPage=48&keywords='

In [5]:
def get_rightmove_results(url: str, test: bool) -> list:
    """Takes a rightmove query URL and returns a list of
    property listing URLs."""
 
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
    }
    search = requests.get(url, headers=headers)
    soup = bs(search.content, "html.parser")

    # Get the number of results from this query and find the number of
    # pages the query returns. First function returns a page with 48 results.
    result_count = int(soup.find("span", class_="searchHeader-resultCount").get_text().replace(',',''))
    page_count = int((result_count / 48))
    # Ensure we're capturing the last pages of results.
    page_count += 1 if result_count % 48 > 0 else 0

    # Iterate through each page and fetch the URL
    property_links = []
   
    print(f"Scraping {page_count} pages for {result_count} results.")
    for page in range(0, page_count):
        
        if page == 0:
            pass
        else:
            url = extract_rightmove_url(index=page * 48)
            page = requests.get(url, headers=headers)
            soup = bs(page.content, "html.parser")
            time.sleep(random.uniform(2, 8))
            

        # Grabbing all property cards and slicing off the first listing, which is
        # always a "featured property" and may not be relevant to our search
        property_cards = soup.find_all("div", class_="l-searchResult is-list")
        property_cards = property_cards[1:]

        for card in property_cards:
            property_link = card.find(
                "a", class_="propertyCard-priceLink propertyCard-salePrice"
            ).attrs["href"]
            print(property_link)
            property_links.append(property_link)
        if test:
            break
    
    return property_links

property_links_list = get_rightmove_results(url, False)

Scraping 2 pages for 61 results.
/properties/144847202#/?channel=RES_BUY
/properties/144847076#/?channel=RES_BUY
/properties/144850859#/?channel=RES_BUY
/properties/144054590#/?channel=RES_BUY
/properties/144847745#/?channel=RES_BUY
/properties/144848324#/?channel=RES_BUY
/properties/144207782#/?channel=RES_BUY
/properties/139752203#/?channel=RES_BUY
/properties/142600631#/?channel=RES_BUY
/properties/144849602#/?channel=RES_BUY
/properties/144849854#/?channel=RES_BUY
/properties/144850586#/?channel=RES_BUY
/properties/144642431#/?channel=RES_BUY
/properties/144849083#/?channel=RES_BUY
/properties/144850346#/?channel=RES_BUY
/properties/144846971#/?channel=RES_BUY
/properties/144849980#/?channel=RES_BUY
/properties/144846977#/?channel=RES_BUY
/properties/144851645#/?channel=RES_BUY
/properties/144266021#/?channel=RES_BUY
/properties/144846962#/?channel=RES_BUY
/properties/144850865#/?channel=RES_BUY
/properties/144850868#/?channel=RES_BUY
/properties/144846731#/?channel=RES_BUY
/proper

In [6]:
len(property_links_list)

61

In [7]:
def scrape_page(url_suffixs: list) -> list:
    """Scrapes a list of rightmove pages for data. Returns data in the form of a list of dictionaries."""

    # used for rate limiting at the bottom of the for loop
    index = 0

    data_rows = []
    for suffix in url_suffixs:

        
        url_base = 'https://www.rightmove.co.uk'
        url = url_base + suffix

        headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
        }

        try:
            page = requests.get(url, headers=headers, timeout=40)
        except requests.ConnectionError:
            t = random.uniform(30, 90)
            print(f"CONNECTION ERROR. Waiting {t} seconds.")
            time.sleep(t)
            page = requests.get(url, headers=headers, timeout=40)
        
        soup = bs(page.content, 'html.parser')

        # Assigning an ID to the property using the url 
        id = re.findall("([0-9]{7,15})", url)[0]
        id = int(id)

        # extracting the property listing description
        data = soup.find_all('div', class_="STw8udCxUaBUMfOOZu0iL _3nPVwR0HZYQah5tkVJHFh5")
        description = data[0].text

        # Extracting a JSON array from the HTML variable PAGE_MODEL, which contains much of the data we require
        script_elements = soup.find_all('script')
        page_model = [script.text for script in script_elements if 'PAGE_MODEL =' in script.text][0]
        data = page_model.split("PAGE_MODEL = ", 1)[1].strip()
        data = json.loads(data)
        data = data.get('propertyData')

        # Using the JSON array to extract a listing type (typically either added or reduced) and the date the listing was made
        listing = data.get("listingHistory", {}).get('listingUpdateReason', None)
        if listing != None:
            listing = listing.split(' ')
            listing_type = listing[0]
        else:
            listing_type = listing
        
        try:
            if listing[-1] == "today":
                Date = datetime.now().strftime("%Y-%m-%d")
            elif listing[-1] == "yesterday":
                Date = datetime.now() - timedelta(days=1)
                Date = Date.strftime("%Y-%m-%d")
            else:
                Date = listing[-1]
                Date = datetime.strptime(Date, "%d/%m/%Y").strftime("%Y-%m-%d")
        except:
            Date = None
            
        # Using the JSON array to extract more data
        price = data.get("prices", {}).get('primaryPrice', np.nan)
        price = int(re.sub('[£,]', '', price))
        
        address = data.get("address", {}).get('displayAddress')
        outcode = data.get("address", {}).get('outcode')
        incode = data.get("address", {}).get('incode')

        estate_agent = data.get('customer', {}).get('companyName')

        nearest_stations = data.get('nearestStations')

        try:
            nearest_station = nearest_stations[0].get('name')
            distance_from_nearest_station_miles = round(nearest_stations[0].get('distance',np.nan),2)
        except:
            nearest_station = None
            distance_from_nearest_station_miles = np.nan

        try:
            second_nearest_station = nearest_stations[1].get('name')
            distance_from_second_nearest_station_miles = round(nearest_stations[1].get('distance', np.nan),2)
        except:
            second_nearest_station = None
            distance_from_second_nearest_station_miles = np.nan

        try:
            third_nearest_station = nearest_stations[2].get('name')
            distance_from_third_nearest_station_miles = round(nearest_stations[2].get('distance', np.nan),2)
        except:
            third_nearest_station = None
            distance_from_third_nearest_station_miles = np.nan

        bedrooms = data.get('bedrooms', np.nan)
        if bedrooms == None:
            bedrooms = 0


        bathrooms = data.get('bathrooms', np.nan)
        if bathrooms == None:
            bathrooms = np.nan

        size = np.nan
        for unit in data.get('sizings', ()):
            if unit['unit'] == 'sqm':
                size = unit['minimumSize']
        
        tenure = data.get('tenure',{})
        tenure_type = tenure.get('tenureType')
        lease_length = tenure.get('yearsRemainingOnLease', np.nan)
        if lease_length == None or lease_length == 0 or lease_length =='0':
            lease_length = np.nan
       

        living_costs = data.get('livingCosts', {})
        ground_rent = living_costs.get('annualGroundRent')
        if ground_rent == None:
            ground_rent = np.nan

        service_charge = living_costs.get('annualServiceCharge')
        if service_charge == None:
            service_charge = np.nan

        property_type = 'N/A'
        try:
            property_type = [item['primaryText'] for item in data['infoReelItems'] if item['type'].lower() == 'property_type'][0]
        except:
            pass

        # Creating a dictionary using the variables assigned above. For each loop, a dictionary is created. Each dictionary represents a row in the dataframe
        # created in the clean() function below.
        row = {
            "id": id,
            # "testing": testing_keys,
            "Property_Link": url,
            "Address": address,
            "Outcode": outcode,
            "Incode": incode, 
            "Price": price,
            "Listing_Type": listing_type, # typically either added or reduced
            "Date": Date,
            "Property_Type": property_type, # don't really need this one tbh - could be fun to do analyses based on this though 
            "Size_Sqm": size, 
            "Bedrooms": bedrooms,
            "Bathrooms": bathrooms,
            "Ground_Rent": ground_rent,
            "Service_Charge": service_charge,
            "Tenure_Type": tenure_type,
            "Lease_Length_Years": lease_length,
            "Estate_Agent": estate_agent,
            "Nearest_Station": nearest_station,
            "Distance_From_Nearest_Station_Miles": distance_from_nearest_station_miles,
            "Second_Nearest_Station": second_nearest_station,
            "Distance_From_Second_Nearest_Station_Miles": distance_from_second_nearest_station_miles,
            "Third_Nearest_Station": third_nearest_station,
            "Distance_From_Third_Nearest_Station_Miles": distance_from_third_nearest_station_miles,
            "Description": description 
        }

        data_rows.append(row)

        # rate limiting:
        t  = random.uniform(2,5)
        if index % 50 == 0 and index != 0:
            t = t * 5
            print(f"Sleeping {t:.2f} seconds")
            time.sleep(t)
        elif index % 6 == 0 and index != 0:
            print(f"Sleeping {t:.2f} seconds")
            time.sleep(t)       
        index += 1

    return data_rows 

json_results = scrape_page(property_links_list)
json_results




Sleeping 2.88 seconds
Sleeping 3.64 seconds
Sleeping 3.63 seconds
Sleeping 4.91 seconds
Sleeping 2.45 seconds
Sleeping 3.96 seconds
Sleeping 2.31 seconds
Sleeping 2.75 seconds
Sleeping 19.07 seconds
Sleeping 3.34 seconds
Sleeping 3.63 seconds


[{'id': 144847202,
  'Property_Link': 'https://www.rightmove.co.uk/properties/144847202#/?channel=RES_BUY',
  'Address': 'Moxon Street, London, W1U',
  'Outcode': 'W1U',
  'Incode': '4EY',
  'Price': 6000000,
  'Listing_Type': 'Added',
  'Date': '2024-02-17',
  'Property_Type': 'Apartment',
  'Size_Sqm': 130,
  'Bedrooms': 2,
  'Bathrooms': 2,
  'Ground_Rent': nan,
  'Service_Charge': nan,
  'Tenure_Type': 'LEASEHOLD',
  'Lease_Length_Years': 998,
  'Estate_Agent': 'PROPERLY PARTNERS LTD',
  'Nearest_Station': 'Baker Street Station',
  'Distance_From_Nearest_Station_Miles': 0.33,
  'Second_Nearest_Station': "Regent's Park Station",
  'Distance_From_Second_Nearest_Station_Miles': 0.39,
  'Third_Nearest_Station': 'Bond Street Station',
  'Distance_From_Third_Nearest_Station_Miles': 0.38,
  'Description': 'A stunning two-bedroom apartment with a study in the heart of Marylebone is finished to the highest specification. It is on the second floor of a brand-new development boasting contempo

In [8]:
len(json_results)

61

In [9]:
def clean(rows_list: list) -> pd.DataFrame:
    """Takes a list of property row dictionaries, returns a dataframe with a correct schema.
    Depends on dtypes.yaml."""

    df = pd.DataFrame(rows_list)
    with open("./dtypes.yaml", "rb") as schema_yaml:
        schema = safe_load(schema_yaml)["raw_dtypes"] 

    try:
        df = df.astype(schema)
    except BaseException as error:
        today = datetime.today().strftime("%Y-%m-%d")
        df.to_csv(f"/tmp/{today}_failed.csv")
        gcs_block = GcsBucket.load('london-properties')
        gcs_block.upload_from_path(
            from_path=f"/tmp/{today}_failed.csv",
            to_path=f"raw_daily_data/failed/{today}_daily_london_failed.csv",
        )
        raise Exception("The dataframe doesn't conform to the specified schema. Dataframe has been loaded to 'failed' directory in the GCS bucket.")

    return df

clean(json_results)

,id,Property_Link,Address,Outcode,Incode,Price,Listing_Type,Date,Property_Type,Size_Sqm,...,Tenure_Type,Lease_Length_Years,Estate_Agent,Nearest_Station,Distance_From_Nearest_Station_Miles,Second_Nearest_Station,Distance_From_Second_Nearest_Station_Miles,Third_Nearest_Station,Distance_From_Third_Nearest_Station_Miles,Description
0,144847202,https://www.rightmove.co.uk/properties/1448472...,"Moxon Street, London, W1U",W1U,4EY,6000000,Added,2024-02-17,Apartment,130.0,...,LEASEHOLD,998.0,PROPERLY PARTNERS LTD,Baker Street Station,0.33,Regent's Park Station,0.39,Bond Street Station,0.38,A stunning two-bedroom apartment with a study ...
1,144847076,https://www.rightmove.co.uk/properties/1448470...,"Moxon Street, London, W1U",W1U,4EY,5165000,Added,2024-02-17,Apartment,128.0,...,LEASEHOLD,998.0,PROPERLY PARTNERS LTD,Baker Street Station,0.33,Regent's Park Station,0.39,Bond Street Station,0.38,This spacious two-bedroom home in the heart of...
2,144850859,https://www.rightmove.co.uk/properties/1448508...,"Flood Street, Chelsea, SW3",SW3,5ST,4250000,Added,2024-02-18,House,214.0,...,FREEHOLD,NaN,Stanley Chelsea,Sloane Square Station,0.54,South Kensington Station,0.56,Gloucester Road Station,0.86,Fantastic Chelsea Freehold HouseThis well-prop...
3,144054590,https://www.rightmove.co.uk/properties/1440545...,"Eton Avenue, London, NW3",NW3,3ET,2750000,Reduced,2024-02-18,Flat,172.0,...,SHARE_OF_FREEHOLD,996.0,KIRE UN LIMITED,Swiss Cottage Station,0.19,South Hampstead Station,0.41,Finchley Road Station,0.46,Have my mum as your neighbour! There are many ...
4,144847745,https://www.rightmove.co.uk/properties/1448477...,"Orchard Grove, London, SW20",SW20,0SE,2000000,Added,2024-02-17,Detached,NaN,...,FREEHOLD,NaN,Knight Frank,Raynes Park Station,0.39,Wimbledon Chase Station,0.86,Dundonald Road Tram Stop,1.08,Located in a secluded and sought after West Wi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,132471695,https://www.rightmove.co.uk/properties/1324716...,"Shepherd Court, Grundy Street, London, E14",E14,6DP,300000,Reduced,2024-02-17,Apartment,NaN,...,LEASEHOLD,242.0,Life Residential,All Saints Station,0.28,Langdon Park Station,0.32,West India Quay Station,0.35,"Guide Price £300,000-£325,000Fantastic chain f..."
57,142018238,https://www.rightmove.co.uk/properties/1420182...,"Building 22, Cadogan Road, Royal Arsenal SE18",SE18,6YL,290000,Reduced,2024-02-17,Apartment,NaN,...,LEASEHOLD,NaN,Royal Arsenal Residential,Woolwich Station,0.13,Woolwich Arsenal Station,0.30,Plumstead Station,0.58,"Contemporary one bed apartment, in an attrac..."
58,141001691,https://www.rightmove.co.uk/properties/1410016...,"Camberwell Road, London, SE5",SE5,0EN,290000,Reduced,2024-02-17,Apartment,41.0,...,LEASEHOLD,170.0,Laurels Estate Agents Limited,Kennington Station,0.58,Oval Station,0.80,Elephant & Castle Station,0.90,"Nestled within a delightful period building, t..."
59,144850811,https://www.rightmove.co.uk/properties/1448508...,"Deals Gateway, London, SE13",SE13,7QE,275000,Added,2024-02-18,Flat,NaN,...,LEASEHOLD,NaN,Strike,Deptford Bridge Station,0.16,St. John's Station,0.23,Elverson Road Station,0.30,Strike is pleased to present this extremely we...
